## 3장. 시카고 샌드위치 맛집 분석

### 1. 시카고 샌드위치 맛집 소개 사이트에 접근하기

In [ ]:
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen

url_base = "https://www.chicagomag.com"
url_sub = '/chicago-magazine/november-2012/best-sandwiches-chicago/'
url = url_base + url_sub

req = Request(url, headers={'User-Agent':'Mozilla/5.0'})
html = urlopen(req)
soup = BeautifulSoup(html, "html.parser")


In [ ]:
len(soup.find_all('div', 'sammy'))

50

50개의 맛집을 정확히 찾음을 확인할 수 있다.

In [ ]:
print(soup.find_all('div', 'sammy')[0])

<div class="sammy" style="position: relative;">
<div class="sammyRank">1</div>
<div class="sammyListing"><a href="/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Old-Oak-Tap-BLT/"><b>BLT</b><br/>
Old Oak Tap<br/>
<em>Read more</em> </a></div>
</div>


첫번째 맛집이 Old Oak Tap의 BLT임을 확인할 수 있다.

### 2. 접근한 웹 페이지에서 원하는 데이터 추출하고 정리하기

In [ ]:
tmp_one =soup.find_all('div', 'sammy')[0]
type(tmp_one)

bs4.element.Tag

bs4.element.Tag형태는 그 변수에 다시 태그로 찾는 (find, find_all) 명령을 사용할 수 있다.

In [ ]:
tmp_one.find(class_= 'sammyRank')

<div class="sammyRank">1</div>

find 명령을 사용하고 sammyRank를 찾아보면 나타난다.  여기서 text만 취하면 된다.

In [ ]:
tmp_one.find(class_="sammyRank").get_text()

'1'

랭크를 추출할 수 있다.

In [ ]:
tmp_one.find(class_='sammyListing').get_text()

'BLT\nOld Oak Tap\nRead more '

가게 이름과 메뉴이름이 함께 추출되었다.

In [ ]:
tmp_one.find('a')['href']

'/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Old-Oak-Tap-BLT/'

a 태그에서 href 정보를 가지고 클릭했을 때 연결될 주소도 저장할 수 있다.

In [ ]:
import re

tmp_string = tmp_one.find(class_='sammyListing').get_text()

re.split(('\n|\r\n'),tmp_string)

print(re.split(('\n|\r\n'),tmp_string)[0])
print(re.split(('\n|\r\n'),tmp_string)[1])

BLT
Old Oak Tap


정규식을 사용하기 위해 re를 import했다.
첫 번째 추출되는 내용은 메뉴, 두 번째로 추출되는 내용은 가게이름이다.

In [ ]:
import urllib.parse

rank=[]
main_menu=[]
cafe_name=[]
url_add=[]

list_soup = soup.find_all('div', 'sammy')

for item in list_soup:
  rank.append(item.find(class_='sammyRank').get_text())
  tmp_string = item.find(class_='sammyListing').get_text()

  main_menu.append(re.split(('\n|\r\n'),tmp_string)[0])
  cafe_name.append(re.split(('\n|\r\n'),tmp_string)[1])

  url_add.append(urllib.parse.urljoin(url_base, item.find('a')['href']))

print(rank[:5])
print(main_menu[:5])
print(cafe_name[:5])
print(url_add[:5])

['1', '2', '3', '4', '5']
['BLT', 'Fried Bologna', 'Woodland Mushroom', 'Roast Beef', 'PB&L']
['Old Oak Tap', 'Au Cheval', 'Xoco', 'Al’s Deli', 'Publican Quality Meats']
['https://www.chicagomag.com/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Old-Oak-Tap-BLT/', 'https://www.chicagomag.com/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Au-Cheval-Fried-Bologna/', 'https://www.chicagomag.com/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Xoco-Woodland-Mushroom/', 'https://www.chicagomag.com/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Als-Deli-Roast-Beef/', 'https://www.chicagomag.com/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Publican-Quality-Meats-PB-L/']


In [ ]:
len(rank), len(main_menu), len(cafe_name), len(url_add)


(50, 50, 50, 50)

모두 각각 다 저장이 잘 되었음을 알 수 있다.

In [ ]:
import pandas as pd

data = {'Rank':rank, 'Cafe':cafe_name, 'Menu':main_menu, 'URL':url_add}
df = pd.DataFrame(data)
df.head()

,Rank,Cafe,Menu,URL
0,1,Old Oak Tap,BLT,https://www.chicagomag.com/Chicago-Magazine/No...
1,2,Au Cheval,Fried Bologna,https://www.chicagomag.com/Chicago-Magazine/No...
2,3,Xoco,Woodland Mushroom,https://www.chicagomag.com/Chicago-Magazine/No...
3,4,Al’s Deli,Roast Beef,https://www.chicagomag.com/Chicago-Magazine/No...
4,5,Publican Quality Meats,PB&L,https://www.chicagomag.com/Chicago-Magazine/No...


### 3. 다수의 웹 페이지에 자동으로 접근해서 원하는 정보 가져오기

In [ ]:
df['URL'][0]

'https://www.chicagomag.com/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Old-Oak-Tap-BLT/'

In [ ]:
req_new = Request(df['URL'][0], headers={'User-Agent':'Mozilla/5.0'})
html_new = urlopen(req_new)

soup_tmp_new = BeautifulSoup(html_new, "html.parser")

In [ ]:
print(soup_tmp_new.find('p', 'addy'))

<p class="addy">
<em>$10. 2109 W. Chicago Ave., 773-772-0406, <a href="http://www.theoldoaktap.com/">theoldoaktap.com</a></em></p>


In [ ]:
price_tmp = soup_tmp_new.find('p', 'addy').get_text()
price_tmp

'\n$10. 2109 W. Chicago Ave., 773-772-0406, theoldoaktap.com'

In [ ]:
price_tmp.split()

['$10.', '2109', 'W.', 'Chicago', 'Ave.,', '773-772-0406,', 'theoldoaktap.com']

In [ ]:
price_tmp.split()[0]

'$10.'

In [ ]:
price_tmp.split()[0][:-1]

'$10'

In [ ]:
' '.join(price_tmp.split()[1:-2])

'2109 W. Chicago Ave.,'

In [ ]:
from tqdm import tqdm_notebook

price = []
address= []

for n in tqdm_notebook(df.index):
  req_new2 = Request(df['URL'][n], headers={'User-Agent':'Mozilla/5.0'})
  html_new2 = urlopen(req_new2)
  soup_tmp_new2 = BeautifulSoup(html_new2, 'lxml')
  
  gettings = soup_tmp_new2.find('p', 'addy').get_text()

  price.append(gettings.split()[0][:-1])
  address.append(' '.join(gettings.split()[1:-2]))

print(price)
print(address)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


  0%|          | 0/50 [00:00<?, ?it/s]

['$10', '$9', '$9.50', '$9.40', '$10', '$7.25', '$16', '$10', '$9', '$17', '$11', '$5.49', '$14', '$10', '$13', '$4.50', '$11.95', '$11.50', '$6.25', '$15', '$5', '$6', '$8', '$5.99', '$7.52', '$11.95', '$7.50', '$12.95', '$7', '$21', '$9.79', '$9.75', '$13', '$7.95', '$9', '$9', '$8', '$8', '$7', '$6', '$7.25', '$11', '$6', '$9', '$5.49', '$8', '$6.50', '$7.50', '$8.75', '$6.85']
['2109 W. Chicago Ave.,', '800 W. Randolph St.,', '445 N. Clark St.,', '914 Noyes St., Evanston,', '825 W. Fulton Mkt.,', '100 E. Walton', '1639 S. Wabash Ave.,', '2211 W. North Ave.,', '3619 W. North Ave.,', '3267 S. Halsted St.,', '2537 N. Kedzie Blvd.,', 'Multiple', '3124 N. Broadway,', '3455 N. Southport Ave.,', '2657 N. Kedzie Ave.,', '1120 W. Grand Ave.,', '1141 S. Jefferson St.,', '333 E. Benton Pl.,', '1411 N. Wells St.,', '1747 N. Damen Ave.,', '3209 W. Irving Park', 'Multiple', '5347 N. Clark St.,', '2954 W. Irving Park Rd.,', 'Multiple', '191 Skokie Valley Rd., Highland Park,', 'Multiple', '1818 W.

In [ ]:
len(price), len(address), len(df)

(50, 50, 50)

가격과 주소 정보가 잘 저장되었음을 확인할 수 있다.

In [ ]:
df = df.assign(Price = price, Address=address)

df.set_index('Rank', inplace=True)
df.head()

,Cafe,Menu,URL,Price,Address
Rank,,,,,
1,Old Oak Tap,BLT,https://www.chicagomag.com/Chicago-Magazine/No...,$10,"2109 W. Chicago Ave.,"
2,Au Cheval,Fried Bologna,https://www.chicagomag.com/Chicago-Magazine/No...,$9,"800 W. Randolph St.,"
3,Xoco,Woodland Mushroom,https://www.chicagomag.com/Chicago-Magazine/No...,$9.50,"445 N. Clark St.,"
4,Al’s Deli,Roast Beef,https://www.chicagomag.com/Chicago-Magazine/No...,$9.40,"914 Noyes St., Evanston,"
5,Publican Quality Meats,PB&L,https://www.chicagomag.com/Chicago-Magazine/No...,$10,"825 W. Fulton Mkt.,"


### 4. 맛집 위치를 지도에 표기하기

In [ ]:
import folium
import googlemaps
import numpy as np

gmaps = googlemaps.Client(key=gmaps_key)

In [ ]:
lat =[]
lng =[]

for n in tqdm_notebook(df.index):
  if df['Address'][n] != "Multiple":
    target_name = df['Address'][n] + ', '+'Chicago'
    gmaps_output=gmaps.geocode(target_name)
    location_output = gmaps_output[0].get('geometry')
    lat.append(location_output['location']['lat'])
    lng.append(location_output['location']['lng'])
  else:
    lat.append(np.nan)
    lng.append(np.nan)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  after removing the cwd from sys.path.


  0%|          | 0/50 [00:00<?, ?it/s]

In [ ]:
df['lat'] = lat
df['lng'] = lng

df.head()

,Cafe,Menu,URL,Price,Address,lat,lng
Rank,,,,,,,
1,Old Oak Tap,BLT,https://www.chicagomag.com/Chicago-Magazine/No...,$10,"2109 W. Chicago Ave.,",41.895558,-87.679967
2,Au Cheval,Fried Bologna,https://www.chicagomag.com/Chicago-Magazine/No...,$9,"800 W. Randolph St.,",41.884639,-87.647590
3,Xoco,Woodland Mushroom,https://www.chicagomag.com/Chicago-Magazine/No...,$9.50,"445 N. Clark St.,",41.890523,-87.630783
4,Al’s Deli,Roast Beef,https://www.chicagomag.com/Chicago-Magazine/No...,$9.40,"914 Noyes St., Evanston,",42.058322,-87.683748
5,Publican Quality Meats,PB&L,https://www.chicagomag.com/Chicago-Magazine/No...,$10,"825 W. Fulton Mkt.,",41.886604,-87.648536


In [ ]:
mapping = folium.Map(location=[df['lat'].mean(), df['lng'].mean()], zoom_start=11)
folium.Marker([df['lat'].mean(), df['lng'].mean()], popup='center').add_to(mapping)

mapping

In [ ]:
mapping = folium.Map(location=[df['lat'].mean(), df['lng'].mean()], zoom_start=11)

for n in df.index:
  if df['Address'][n] != 'Multiple':
    folium.Marker([df['lat'][n], df['lng'][n]], popup=df['Cafe'][n]).add_to(mapping)

mapping